<a href="https://colab.research.google.com/github/ngmars/Face-Mask-CNN/blob/master/Face_Mask_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Libraries


In [ ]:
import os
import tensorflow as tf
import cv2
import numpy as np
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.layers import Dense, Input, Dropout, Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3


Training data preprocessing

In [ ]:
data_path= r'/content/drive/My Drive/Colab Notebooks/Training Set'
categories= os.listdir(data_path)
labels= [i for i in range(len(categories))]

label_dict= dict(zip(categories,labels))

print (label_dict)
for expression in os.listdir("/content/drive/My Drive/Colab Notebooks/Training Set"):
  print(str(len(os.listdir("/content/drive/My Drive/Colab Notebooks/Training Set/"+ expression)))+ " " + expression + " Images")

In [ ]:
"""
data= []
target= []
for category in categories:
    folder_path= os.path.join(data_path, category)
    img_names=os.listdir(folder_path)
    
    for img_name in img_names:
        img_path=os.path.join(folder_path, img_name)
        img= cv2.imread(img_path)
        
        try:
            resized= cv2.resize(img,(150,150))
            data.append(resized)
            target.append(label_dict[category])
        except Exception as e:
            print(e)"""

img_size= 150
batch_size =64


datagen_train= ImageDataGenerator(horizontal_flip=True)

train_data = datagen_train.flow_from_directory("/content/drive/My Drive/Colab Notebooks/Training Set/", 
                                                    target_size=(img_size,img_size),
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True)

datagen_test= ImageDataGenerator(horizontal_flip=True)
test_data = datagen_test.flow_from_directory("/content/drive/My Drive/Colab Notebooks/Test_set/", 
                                                    target_size=(img_size,img_size),
                                                    color_mode='grayscale',
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True)


Pre-trained Model


In [ ]:
pre_trained_model= InceptionV3(input_shape= (150,150,3),
                              include_top= False,
                              weights= None)
pre_trained_model.load_weights(r'/content/drive/My Drive/Colab Notebooks/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')

for layer in pre_trained_model.layers:
    layer.trainable= True

pre_trained_model.summary()

In [ ]:

last_layer= pre_trained_model.get_layer('mixed8')
print(last_layer.output_shape)
last_output= last_layer.output

Building our model

In [ ]:

from tensorflow.keras.optimizers import RMSprop

#Flatten output layer 1D
x= layers.Flatten()(last_output)


#Fully connected layer
x= layers.Dense(1024)(x)
x= layers.BatchNormalization()(x)
x= layers.Activation('relu')(x)
#Dropout due to overfitting
x= layers.Dropout(0.2)(x)

#Sigmoid for classification
x= layers.Dense(2, activation='softmax')(x)

model = Model(pre_trained_model.input, x)

model.compile(optimizer = Adam(lr=0.0005), 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

model.summary()

Save Model


In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau 
"""
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)
model.save_weights(checkpoint_path.format(epoch=10))
"""
epochs =1
steps_per_epoch= train_data.n//train_data.batch_size
test_steps=test_data.n//test_data.batch_size

checkpoint= ModelCheckpoint("model", monitor='val_accuracy', save_weights_only=True,
                            mode='max', verbose=1)
reduce_lr= ReduceLROnPlateau(monitor='val_loss', factor= 0.1, patience=2, model='auto',min_lr=0.00001)
callbacks= [ checkpoint, reduce_lr]

history = model.fit(x= train_data, steps_per_epoch= steps_per_epoch, epochs= epochs, validation_data= test_data,validation_steps=test_steps, callbacks= callbacks)